In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, SparsePauliOp
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from qiskit.circuit import Parameter, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFTGate
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la

# Jordan Lee Preskill encoding

Discretize
\begin{equation}
x_{\ell} = - x_{\text{max}} + \ell \delta_x\,, \quad \delta_x = \frac{2 x_{\text{max}}}{2^{n_q} - 1}\,, \quad \ell \in [0, 2^{n_q} - 1]
\end{equation}
Eigenstates of $\hat{x}$ are now represented by the ket representing the bitstring $(\ell)_2$. For example,
\begin{equation}
\hat{x} \ket{\ell} = ( - x_{\text{max}} + \ell \delta_x ) \ket{\ell}
\end{equation}
It can be shown that this can be implemented on the Hilbert space of qubits with the following operator
\begin{equation}
\hat{x} =  -\frac{x_{\text{max}}}{2^{n_q} - 1} \sum_{\ell = 0}^{n_q - 1}2^{\ell}Z_{\ell}
\end{equation}
The conjugate momenta take the values
\begin{equation}
k_{x} = - \frac{\pi}{\delta_x} + \left(\ell + \frac{1}{2}\right) \frac{2 \pi}{2^{n_q} \delta_x}
\end{equation}
The conjugate momentum operator can be implemented as (after doing a Fourier transform)
\begin{equation}
\hat{p} = -\frac{\pi}{2^{n_q}\delta_x}\sum_{\ell = 0}^{n_q - 1} 2^{\ell} Z_{\ell}
\end{equation}

The first thing that we should do is figure out what $x^2$ looks like
$$\hat{x}^2 = \left(\frac{x_{\text{max}}}{2^{n_q} -1}\right)^2 \sum_{\ell, k = 0}^{n_q - 1} 2^{\ell}2^{k} Z_{\ell} Z_k$$
Sum over $\ell < k$. The $\ell = k$ terms are just identity and therefore just a phase, so they don't contribute to dyanmics. Essentially this oeprator is just

\begin{equation}
\hat{x}^2 = 2 \times \left(\frac{x_{\text{max}}}{2^{n_q} -1}\right)^2 \sum_{\ell < k}^{n_q - 1} 2^{\ell}2^{k} Z_{\ell} Z_k
\end{equation}

Create the evolution operator $\exp(- i \delta t \hat{x}^2)$.

In [96]:
def x2_term(qc, m_ctr, delta_t, x_max):
    qubits = qc.num_qubits
    x_prefactor = - x_max / (2 ** qubits - 1)
    delta_x = 2 * x_max / (2 ** qubits - 1)
    for i in range(qubits):
        for j in range(i + 1, qubits):
            qc.rzz(2 * 2 ** (i + j) * (1 + m_ctr) * x_prefactor ** 2 * delta_t, i, j)

Fourier transform is 
\begin{equation}
\ket{j} \rightarrow \frac{1}{\sqrt{2^N}}\sum_{k = 0}^{2^N - 1} e^{ 2\pi i j k / 2^N} \ket{k}
\end{equation}
But we actually want this
\begin{equation}
\ket{j} \rightarrow \frac{1}{\sqrt{2^N}}\sum_{k = 0}^{2^N - 1} e^{ 2\pi i x_j p_k / 2^N} \ket{k}
\end{equation}
But
\begin{equation}
x_j p_k = ( - x_{\text{max}} + j \delta_x) \times \left(-\frac{\pi}{\delta_x} + \left(k + \frac{1}{2} \right) \frac{2\pi}{2^{n_q} \delta_x}\right)
\end{equation}
This has the `extra terms'
\begin{equation}
x_j p_k = \frac{2\pi j k}{2^{n_q}} + j \pi \left(\frac{1}{2^{n_q}} - 1 \right) + k \pi \left(\frac{1}{2^{n_q}} - 1\right) -\frac{\pi x_{\text{max}}}{2^{n_q}\delta_x}
\end{equation}

\begin{equation}
k_{x} = - \frac{\pi}{\delta_x} + \left(\ell + \frac{1}{2}\right) \frac{2 \pi}{2^{n_q} \delta_x}
\end{equation}
The conjugate momentum operator can be implemented as (after doing a Fourier transform)
\begin{equation}
\hat{p} = -\frac{\pi}{2^{n_q}\delta_x}\sum_{\ell = 0}^{n_q - 1} 2^{\ell} Z_{\ell}
\end{equation}

In [97]:
def p2_term(qc, lam0f, delta_t, x_max):
    qubits = qc.num_qubits
    delta_x = 2 * x_max / (2 ** qubits - 1)
    p_prefactor = - np.pi / 2 ** qubits / delta_x
    alpha = (1 / 2 ** qubits - 1) * np.pi
    for i in range(qubits):
        qc.rz(alpha * 2 ** i, i)
    qc.append(QFTGate(qubits), range(qubits))
    for i in range(qubits):
        qc.rz(alpha * 2 ** i, i)
    for i in range(qubits):
        for j in range(i + 1, qubits):
            qc.rzz(2 * 2 ** (i + j) * lam0f * p_prefactor ** 2 * delta_t, i, j)
    for i in range(qubits):
        qc.rz(- alpha * 2 ** i, i)
    qc.append(QFTGate(qubits).inverse(), range(qubits))
    for i in range(qubits):
        qc.rz(- alpha * 2 ** i, i)

The anharmonic oscillator has the Hamiltonian
\begin{equation}
H = \frac{p^2}{2} + \frac{x^2}{2} + \lambda x^4
\end{equation}
For this we need
\begin{equation}
x^4 = \left(\frac{x_{\text{max}}}{2^{N} - 1}\right)^4 \sum_{i, j, k, l} 2^{i + j + k + l} Z_i Z_j Z_k Z_l
\end{equation}
We can reduce the number of gates applied by noting that the above reduces to identity when all the four sites are equal giving rise to an irrelevant global phase. Moreover, when any two of the site indices are equal, say $i = j$, then we have a two-qubit $Z-Z$ interaction (provided $k \neq l \neq i$).

For the case when all four sites are distinct, we have symmetry under permutations of $(i, j, k, l)$. Therefore, we can write this contribution as
\begin{equation}
\left(\frac{x_{\text{max}}}{2^N - 1}\right)^{4} 4! \sum_{i<j<k<l} 2^{i+j+k+l} Z_iZ_jZ_kZ_l
\end{equation}
Next, when there are two equal indices, we have the following contribution
\begin{equation}
2 \left(\frac{x_{\text{max}}}{2^N - 1}\right)^4 \, {}^4 C_2 \sum_{i = 0}^{N - 1} \sum_{k < l, k \neq i, l \neq i} 2^{2i + k + l} Z_k Z_l
\end{equation}
Finally, when there are three indices equal, we have
\begin{equation}
\left(\frac{x_{\text{max}}}{2^N - 1}\right)^4\, {}^4 C_3 \sum_{i = 0}^{N - 1}\sum_{j \neq i} 2^{3 i + j} Z_i Z_j
\end{equation}

In [121]:
def rz4(qc, theta, i, j, k, l):
    qc.cx(i, j)
    qc.cx(j, k)
    qc.cx(k, l)
    qc.rz(theta, l)
    qc.cx(k, l)
    qc.cx(j, k)
    qc.cx(i, j) # I mistyped this as (k, l) during the tutorial

In [110]:
def x4_term(qc, lam, delta_t, x_max):
    qubits = qc.num_qubits
    x_prefactor = - x_max / (2 ** qubits - 1)
    delta_x = 2 * x_max / (2 ** qubits - 1)
    '''
    Apply the Z_i Z_j Z_k Z_l term: four site interaction
    '''
    for i in range(qubits):
        for j in range(i + 1, qubits):
            for k in range(j + 1, qubits):
                for l in range(k + 1, qubits):
                    coeff = 2 ** (i + j + k + l) * 24 * lam * x_prefactor ** 4
                    rz4(qc, 2 * coeff * delta_t, i, j, k, l)
    '''
    Apply the two-site piece with two and only two equal indices
    '''
    for i in range(qubits):
        for j in range(qubits):
            if j == i:
                continue
            for k in range(j + 1, qubits):
                if k == i:
                    continue
                coeff = 2 ** (2 * i + j + k) * 12 * lam * x_prefactor ** 4 # I wrote 2 * i  + j + l instead of k here in the video
                qc.rzz(2 * coeff * delta_t, j, k)
    '''
    Apply the two qubit interaction where three indices are equal
    '''
    for i in range(qubits):
        for j in range(qubits):
            if j == i:
                continue
            coeff = 2 ** (3 * i + j) * 4 * lam * x_prefactor ** 4
            qc.rzz(2 * coeff * delta_t, i, j)

In [111]:
def strang_step(qc, lam, m_ctr, delta_t, x_max):
    ''' Apply strang split exp(A+B) = exp(A/2) exp(B) exp(A/2)
    A here is x^2 + lambda * x^4 piece
    '''
    # apply x^2 + lam * x^4
    x2_term(qc, m_ctr, delta_t / 2, x_max)
    x4_term(qc, lam, delta_t / 2, x_max)
    # apply the p^2 piece
    p2_term(qc, 1, delta_t, x_max)
    # apply again the x^2 ... piece (this is A / 2)
    x2_term(qc, m_ctr, delta_t / 2, x_max)
    x4_term(qc, lam, delta_t / 2, x_max)

Now we need ramp terms
$$ r(u) = 3u^2 - 2 u^3 $$

In [112]:
# first thing is the ramp
def r(u):
    return u * u * (3.0 - 2.0 * u)

In [113]:
def lamOf(u, lam):
    if 0.0 <= u <= 1.0:
        return lam * r(u)
    if 1.0 < u <= 2:
        return lam
    return 0

# \delta m^2 = - 6 \lambda^2
def dm2Of(u, lam):
    if 0.0 <= u <= 1.0:
        return - 6 * lam * r(u)
    if 1.0 < u <= 2:
        return - 6 * lam * (1 - r(u - 1))
    return 0.0

Need to build the full adiabatic evolution circuit

In [114]:
def build_circuit(lam, num_qubits, x_max, tPrep = 25.0, nSteps = 250):
    '''
    Part A:
    First we need to build the x^2 + p^2 ground state.
    '''
    qc = build_circuit_gs_prep(num_qubits, x_max, 60, 600)
    '''
    Part B:
    Now go to lambda * x^4 ground state with adiabatic state prep
    '''
    dt = tPrep / 2 / nSteps
    for j in range(1, 2 * nSteps + 1):
        u = (j - 0.5) / nSteps
        lam_t = lamOf(u, lam)
        dm_ctr = dm2Of(u, lam)
        strang_step(qc, lam_t, dm_ctr, dt, x_max)
    return qc

In [115]:
# Part A
def build_circuit_gs_prep(num_qubits, x_max, tSteps = 60, nSteps = 600):
    # GS is just exp( - x ** 2 / 2)
    dx = 2 * x_max / (2 ** num_qubits - 1)
    x_values = np.arange(-x_max, x_max + dx, dx)
    gs = np.exp( - x_values ** 2 / 2)
    gs /= np.sqrt(np.sum(np.abs(gs)**2))
    qc = QuantumCircuit(num_qubits)
    qc.initialize(gs)
    return qc

In [116]:
x_max = 10
num_qubits = 5

In [117]:
lam = 5

In [118]:
qc = build_circuit(lam, num_qubits, x_max)

In [119]:
state = Statevector.from_instruction(qc)

# Check with numpy

In [120]:
def jlp_grids(n_qubits: int, x_max: float):
    N  = 2**n_qubits
    dx = 2.0 * x_max / (N - 1)
    j  = np.arange(N, dtype=float)
    # position grid (exactly N samples)
    x_vals = -x_max + j * dx
    # half-shifted momentum grid
    k_vals = -np.pi/dx + (j + 0.5) * (2.0*np.pi)/(N * dx)
    return x_vals, k_vals, dx

def plane_wave_S(n_qubits: int, x_max: float):
    x_vals, k_vals, _ = jlp_grids(n_qubits, x_max)
    N = 2**n_qubits
    S = np.exp(1j * np.outer(k_vals, x_vals)) / np.sqrt(N)
    return S
def jlp_X_P(n_qubits: int, x_max: float):
    x_vals, k_vals, _ = jlp_grids(n_qubits, x_max)
    X = np.diag(x_vals.astype(complex))
    S = plane_wave_S(n_qubits, x_max)
    P = S.conj().T @ np.diag(k_vals.astype(complex)) @ S
    return X, P

def jlp_H(n_qubits: int, x_max: float, lam: float, m: float):
    X, P = jlp_X_P(n_qubits, x_max)
    X2 = X @ X
    return 0.5*(P@P) + 0.5*(1.0+m)*X2 + lam*(X2@X2)

def exact_diagonalize(n_qubits: int, x_max: float, lam: float, m: float):
    H = jlp_H(n_qubits, x_max, lam, m)
    evals, evecs = la.eigh(H)   # columns are eigenvectors
    return evals, evecs, H
if __name__ == "__main__":
    n_q   = num_qubits
    x_max = 10
    # lam   = 0.0  # no lambda here we are looking at the SHO ground state
    m     = 0.0

    evals, evecs, H = exact_diagonalize(n_q, x_max, lam, m)
    print("Lowest 6 eigenvalues:", np.round(evals[:6], 8))

    # compare your Gaussian initializer to the discrete ground state if you want:
    N = 2**n_q
    x_vals, _, _ = jlp_grids(n_q, x_max)
    gaussian = np.exp(-0.5 * x_vals**2).astype(complex)
    gaussian /= np.linalg.norm(gaussian)

    v0 = evecs[:, 0]
    overlap = float(abs(np.vdot(v0, state))**2)
    print("Overlap(gaussian, discrete ground):", overlap)

Lowest 6 eigenvalues: [ 1.21122209  4.47762461  8.81071697 10.78596992 39.10778585 39.36277487]
Overlap(gaussian, discrete ground): 0.9972657022576378
